<a href="https://colab.research.google.com/github/bvanitsthian/photocircle_nlp/blob/main/create_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Project Objectives**

1. Create tags for users who align with these types of products and services:
Homeowner services & products
Home improvement & remodeling
Home and auto insurance
Home goods/furniture
Home warranty
Solar panels
Travel tours
Sorority apparel

2. Understand user base more.
Who is most active?
Who is our ideal user?
With more data in the future, how can we figure out which segment is growing fastest?

# **Import Libraries and Download nltk Packages**

In [ ]:
# Run this cell to keep the notebook active
import time

while True:
    print(time.ctime())
    time.sleep(60)

In [ ]:
!pip install emoji
!pip install spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

In [ ]:
import pandas as pd
import re
import emoji
import regex
import spacy
from IPython.display import display
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
pd.set_option('display.max_columns', 200)

#**USERS**

#**Identify M/F in Users**

In [ ]:
users = pd.read_csv('users.csv', usecols=range(7))

<ipython-input-5-294014fac6d9>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('users.csv', usecols=range(7))


In [ ]:
users.head(3)

,Unnamed: 0,user_uuid,display_name,email_domain,active_count_in_last_30_days,media_added_count,first_name
0,0.0,00000000-4f80-9c33-7b70-8c4f08000075,Andrew Lazuka,gmail.com,0,0.0,Andrew
1,1.0,00000000-4f81-d43f-7b70-8c4f08001119,Ari Joseph,gmail.com,0,0.0,Ari
2,2.0,00000000-4f81-d7a5-7b70-8c4f08001185,Robert Morgan,backbeat.net,0,0.0,Robert


In [ ]:
# Open the csv file and read its contents
names = pd.read_csv('usa_names.csv')
names = names.rename(columns={'name': 'first_name'})

In [ ]:
names.shape

(1957046, 4)

In [ ]:
names['genders'] = names[['first_name','gender']].groupby(['first_name'])['gender'].transform(lambda x: ','.join(sorted(set(x))))
names[['first_name','genders']].drop_duplicates() #Uh oh some names are uni-sex

,first_name,genders
0,Mary,"F,M"
1,Shirley,"F,M"
2,Barbara,"F,M"
3,Betty,"F,M"
4,Patricia,"F,M"
...,...,...
1956841,Predrag,M
1956907,Sedwick,M
1956961,Tarvin,M
1956988,Torianno,M


In [ ]:
names.head()

,first_name,gender,num_registered,year,genders
0,Mary,F,54372,1936,"F,M"
1,Shirley,F,35161,1936,"F,M"
2,Barbara,F,31682,1936,"F,M"
3,Betty,F,25866,1936,"F,M"
4,Patricia,F,23914,1936,"F,M"


In [ ]:
names_gender_count = names[['first_name','gender','num_registered']].groupby(['first_name','gender']).sum()

In [ ]:
names_gender_count = names_gender_count.reset_index()

In [ ]:
names_f = names_gender_count[names_gender_count['gender'] == 'F']

In [ ]:
names_f = names_f[['first_name','num_registered']]
names_f = names_f.rename(columns = {'num_registered':'num_female'})
names_f.head()

,first_name,num_female
1,Aabha,35
3,Aabidah,5
5,Aabriella,38
6,Aada,13
10,Aadaya,8


In [ ]:
names_m = names_gender_count[names_gender_count['gender'] == 'M']

In [ ]:
names_m = names_m[['first_name','num_registered']]
names_m = names_m.rename(columns = {'num_registered':'num_male'})
names_m.head()

,first_name,num_male
0,Aaban,114
2,Aabid,16
4,Aabir,10
7,Aadam,273
8,Aadan,130


In [ ]:
users_gender = pd.merge(users, names_f, on='first_name', how='left')
users_gender = pd.merge(users_gender, names_m, on='first_name', how='left')

In [ ]:
users_gender.head(3)

,Unnamed: 0,user_uuid,display_name,email_domain,active_count_in_last_30_days,media_added_count,first_name,num_female,num_male
0,0.0,00000000-4f80-9c33-7b70-8c4f08000075,Andrew Lazuka,gmail.com,0,0.0,Andrew,4900.0,1291279.0
1,1.0,00000000-4f81-d43f-7b70-8c4f08001119,Ari Joseph,gmail.com,0,0.0,Ari,3419.0,14339.0
2,2.0,00000000-4f81-d7a5-7b70-8c4f08001185,Robert Morgan,backbeat.net,0,0.0,Robert,20099.0,4820129.0


# **Fix Null Values in Users Name**

In [ ]:
users_gender['num_female'] = users_gender['num_female'].fillna(0)
users_gender['num_male'] = users_gender['num_male'].fillna(0)

#**Get M / F pct**

In [ ]:
users_gender['female_pct'] = round(users_gender['num_female']/(users_gender['num_female'] + users_gender['num_male'])*100,2)

In [ ]:
users_gender['female_pct'] = users_gender['female_pct'].fillna(50)

In [ ]:
users_gender.head(3)

,Unnamed: 0,user_uuid,display_name,email_domain,active_count_in_last_30_days,media_added_count,first_name,num_female,num_male,female_pct
0,0.0,00000000-4f80-9c33-7b70-8c4f08000075,Andrew Lazuka,gmail.com,0,0.0,Andrew,4900.0,1291279.0,0.38
1,1.0,00000000-4f81-d43f-7b70-8c4f08001119,Ari Joseph,gmail.com,0,0.0,Ari,3419.0,14339.0,19.25
2,2.0,00000000-4f81-d7a5-7b70-8c4f08001185,Robert Morgan,backbeat.net,0,0.0,Robert,20099.0,4820129.0,0.42


In [ ]:
round(len(users_gender[users_gender['female_pct']>50])/len(users_gender)*100,2) # F:M - 60/40

60.98

# **Identify Students Vs Non-Students**

In [ ]:
# Define a function to create a new column
def my_function(x):
    try:
        return x.split('.')[-1] == 'edu'
    except: 
        return False

# Apply the function to create a new column
users_gender['is_student'] = users_gender['email_domain'].apply(lambda x: my_function(x))

In [ ]:
users_gender[users_gender['is_student']].head(3)

,Unnamed: 0,user_uuid,display_name,email_domain,active_count_in_last_30_days,media_added_count,first_name,num_female,num_male,female_pct,is_student
125,125.0,00000000-4fa6-9d00-1953-bb0532003d43,Stephen Troutman,alumni.usc.edu,0,0.0,Stephen,2931.0,858041.0,0.34,True
301,301.0,00000000-4faf-2e55-1953-bb67c900bb7a,Todd Nightingale,alum.mit.edu,0,0.0,Todd,1060.0,281548.0,0.38,True
348,348.0,00000000-4fb7-2f44-1953-bb483e0011b9,P Witt,gmail.edu,0,0.0,P,0.0,0.0,50.00,True


In [ ]:
round(len(users_gender[users_gender['is_student']])/len(users_gender)*100,2) #only 4.55% registered with .edu email domain

4.55

In [ ]:
#users_gender.to_csv('/content/drive/MyDrive/PhotoCircle/users.csv', index=False)

#**CIRCLES**

In [ ]:
# read the CSV file into a pandas DataFrame
circles = pd.read_csv('circles_V0.csv')

<ipython-input-27-9b0e6bee0945>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  circles = pd.read_csv('circles_V0.csv')


In [ ]:
circles.head()

,Unnamed: 0,circle_uuid,title,media_count
0,0,000059e5-7610-d551-c06b-b55e6eb70002,"SC State Fair 2017, 10-16-17 Mon.",566.0
1,1,00005e63-10d0-78bb-8941-89f368410002,New,12.0
2,2,00005bb8-c655-d551-c04b-87d151c20002,TABA💜,557.0
3,3,8cc2946a-ea65-4604-a4f1-f905126655df,ELVIS,264.0
4,4,4a3a2063-c1da-46df-b45f-32c23ad8d39f,BRANDON BAILO SR#01703957,28.0


#**Exploring Emojis in Circle Titles**


In [ ]:
# Define a regular expression pattern to match emojis
# Define a regular expression pattern to match emojis
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [ ]:
circles['title'] = circles['title'].astype(str)

In [ ]:
# Filter the DataFrame to only include rows with emojis in the 'title' column
emojis_present = circles[circles['title'].apply(lambda x: bool(emoji_pattern.search(x)))]

In [ ]:
len(emojis_present)

65508

In [ ]:
emojis_present.head()

,Unnamed: 0,circle_uuid,title,media_count
2,2,00005bb8-c655-d551-c04b-87d151c20002,TABA💜,557.0
16,16,6cefdd89-3704-414e-8ad0-1977e0dbe6d7,It's FALL 🎃🎃 Picture Time 🎃🎃,138.0
47,47,22ff3722-7d7d-4124-b058-2607f67e68e7,Belgium 🇧🇪 MAY '21,45.0
50,50,723a1f3b-7ccb-413c-b0f9-85a5a2524e21,DELIGHT🫶🏼,358.0
57,57,000053b1-92fd-3dab-0506-e1dbb0400002,Beachin ☀️🌊,56.0


In [ ]:
def extract_emojis(text):
    return ''.join(c for c in text if c in emoji.UNICODE_EMOJI)

In [ ]:
# Apply the 'extract_emojis' function to the 'title' column and save the results to a new column
emojis_present['emojis'] = emojis_present['title'].apply(extract_emojis)

In [ ]:
emojis_present.head()

,circle_uuid,title,media_count,has_greek_words,destination_names,emojis
6,000057af-b679-d551-c018-b1832b9b0002,Avery Grace🎀,38685,False,Avery Grace,🎀
47,00005b53-edbf-d551-c02c-59e3229b0002,Grandbabies! 💜,25235,False,Grandbabies,💜
55,26345fe7-3120-4418-8725-5dc77c38481f,AOII Photo Dump📸,24504,False,Photo Dump,📸
64,0000552d-c439-3901-6837-3bb8d0d70002,Jane & Kathleen 🌼🌷🌻,23308,False,Jane,🌼🌷🌻
73,00005666-76c3-8cff-9671-cea13c410002,Cooper 💚 Mason,22209,False,Cooper,💚


#**Add Greek Tags**

In [ ]:
greek_words = {'alpha', 'beta', 'gamma', 'delta', 'epsilon', 'zeta', 'eta', 'theta', 'iota', 'kappa', 
               'lambda', 'mu', 'nu', 'xi', 'omicron', 'pi', 'rho', 'sigma', 'tau', 'upsilon', 'phi', 
               'chi', 'psi', 'omega'}

# Split the 'title' column into words and convert them to lowercase
words = circles['title'].str.lower().str.split()

# Check if any of the words are in the set of Greek words
mask = words.apply(lambda x: bool(set(x) & greek_words))

# Create a new column to indicate whether any Greek words were found
circles['has_greek_words'] = mask

In [ ]:
circles[circles['has_greek_words']].sort_values('media_count', ascending = False).head(10)

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words
1696372,1696370,00005c29-28fb-d551-c026-3dab0de00002,TRI DELTA,40821.0,True
1615972,1615970,00005c14-2c98-d551-c026-3d86f0c70002,CHI OMEGA,35904.0,True
1019895,1019893,00005c01-58f2-d551-c026-3d90a87a0002,ZTA Nu 2018-2019,33652.0,True
769089,769087,00005c60-c177-d551-c02a-05ec4cee0002,Delta Sigma,32716.0,True
3803,3803,00005b4e-5251-d551-c060-c46821020002,Alpha Phi,32709.0,True
792349,792347,00005de8-961b-f8d1-c13a-31fbf07e0002,Asu Chi Omega,31261.0,True
828195,828193,00005c38-e12b-d551-c026-3d6e67ca0002,Alpha Gam,30146.0,True
1514588,1514586,00005a75-eba2-d551-c00b-26e7604c0002,Alpha Phi Eta Lambda,29224.0,True
1141654,1141652,00005af2-65c8-d551-c065-bd38cbad0002,KAPPA DELTA,29210.0,True
144634,144634,00005c3b-dcf4-d551-c026-3d9a266b0002,Sigma Kappa PR,29113.0,True


In [ ]:
len(circles[circles['has_greek_words']]['circle_uuid'].unique())

9439

In [ ]:
round(len(circles[circles['has_greek_words']])/len(circles)*100,2)   #Only 0.50% of the total circles have greek words

0.51

In [ ]:
round(circles[circles['has_greek_words']]['media_count'].sum()/circles['media_count'].sum()*100,2) #Total Greek circles media contributions as % total

3.58

In [ ]:
circles[circles['has_greek_words']]['media_count'].sum() #Total Greek circles media contributions

12240969.0

In [ ]:
circles['media_count'].sum() #Total circles media contributions

341813160.0

In [ ]:
round(circles[circles['has_greek_words']]['media_count'].sum()/circles['media_count'].sum()*100,2)

3.58

# **Add Real estate tags**

In [ ]:
# Define a list of part-of-speech tags for real estate and home improvement-related words
real_estate_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Define a list of real estate and home improvement-related words
real_estate_words = ['house', 'property', 'mortgage', 'market', 'rental', 'floor', 'wall', 'build', 'renovate', 'upgrade', 'remodel', 'refurbish']

# Define the regular expression pattern to extract real estate and home improvement-related keywords
pattern = r'\b(?:%s)\b' % '|'.join(real_estate_words)

# Extract real estate and home improvement-related keywords from the 'title' column and save to a new column called 'real_estate_keywords'
circles['real_estate_keywords'] = circles['title'].str.findall(pattern, flags=re.IGNORECASE)

# Define a function to check if a string contains real estate and home improvement-related tags or keywords
def is_real_estate(title):
    # Extract real estate and home improvement-related keywords from the title
    keywords = re.findall(pattern, title, flags=re.IGNORECASE)
    # Tokenize the title and tag the parts of speech
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    # Check if any real estate and home improvement-related tags or keywords are present in the title
    if any(tag in real_estate_tags and (word in real_estate_words) for word, tag in pos_tags):
        return True
    elif any(word in real_estate_words for word in keywords):
        return True
    return False

# Apply the 'is_real_estate' function to each value in the 'title' column and save the results to a new column called 'is_real_estate'
circles['is_real_estate'] = circles['title'].apply(is_real_estate)

In [ ]:
circles[circles['is_real_estate']].tail(10)

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate
1841609,1841606,80d4a671-ee51-44f2-81f4-a90ff5390de2,2022 - Friends at Nicole’s house,25.0,False,[house],True
1841798,1841795,474ad5ca-e139-495b-b108-b394cb2f83a7,Station house. 05/20/22,5.0,False,[house],True
1844929,1844926,00005d2b-63c0-f8d1-c156-ba3475410002,Lake house Fun,36.0,False,[house],True
1845623,1845620,1504ecc3-39ac-4509-b66e-ce8deb1f837a,passed out on the floor,51.0,False,[floor],True
1846054,1846051,94816b5f-be17-472b-aea9-0385e1dbd7d0,Grammie house inventory,259.0,False,[house],True
1846659,1846656,6c52bd96-2a96-445b-8a72-688fb828b78b,Graciella’s lake house,99.0,False,[house],True
1849879,1849876,7c08dcc8-5ba2-4489-bd83-1166962b5fab,House remodel,16.0,False,"[House, remodel]",True
1850230,1850227,2b7aa475-6c7f-47ae-af09-4b1c80bac5af,The whore house,101.0,False,[house],True
1853404,1853401,00005675-b021-8cff-9671-cee798f50002,house of soul,33.0,False,[house],True
1858091,1858088,c87f3f65-5bf1-46e9-b491-e8246a4ca2f0,Log Cabin house,6.0,False,[house],True


In [ ]:
circles['real_estate_keywords'].apply(tuple).unique()

array([(), ('House',), ('house',), ('Wall',), ('Rental',), ('Market',),
       ('HOUSE',), ('Remodel',), ('Rental', 'House'), ('wall',),
       ('Build',), ('build',), ('Floor',), ('Property',), ('UPGRADE',),
       ('MARKET',), ('remodel',), ('market',), ('floor',), ('upgrade',),
       ('Upgrade',), ('BUILD',), ('build', 'wall'), ('WALL',),
       ('House', 'build'), ('House', 'House'), ('House', 'Build'),
       ('House', 'Remodel'), ('RENTAL',), ('Floor', 'floor'), ('rental',),
       ('FLOOR',), ('Build', 'House'), ('property',), ('house', 'wall'),
       ('Remodel', 'House'), ('house', 'house'), ('House', 'Wall'),
       ('WalL',), ('remodel', 'house'), ('WAll',), ('Floor', 'Remodel'),
       ('Floor', 'Market'), ('House', 'remodel'),
       ('Floor', 'House', 'Remodel'), ('Mortgage',), ('REMODEL',),
       ('House', 'Rental'), ('Wall', 'Build'), ('Rental', 'Property'),
       ('Floor', 'Floor'), ('HouSe',), ('houSe',), ('MORTGAGE',),
       ('Wall', 'Wall'), ('House', 'Remodel',

In [ ]:
len(circles[circles['is_real_estate']]['circle_uuid'].unique())

1103

In [ ]:
round(len(circles[circles['is_real_estate']])/len(circles)*100,2)

0.06

In [ ]:
circles[circles['is_real_estate']]['media_count'].sum()

141542.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_real_estate']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.04

# **Add Solar Tags**

In [ ]:
# Define a list of part-of-speech tags for solar-related words
solar_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Define a list of solar-related words
solar_words = ['solar', 'panel', 'photovoltaic', 'energy', 'power', 'sun', 'renewable', 'electricity', 'grid', 'system']

# Define the regular expression pattern to extract solar-related keywords
pattern = r'\b(?:%s)\b' % '|'.join(solar_words)

# Extract solar-related keywords from the 'title' column and save to a new column called 'solar_keywords'
circles['solar_keywords'] = circles['title'].str.findall(pattern, flags=re.IGNORECASE)

# Define a function to check if a string contains solar-related tags or keywords
def is_solar(title):
    # Extract solar-related keywords from the title
    keywords = re.findall(pattern, title, flags=re.IGNORECASE)
    # Tokenize the title and tag the parts of speech
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    # Check if any solar-related tags or keywords are present in the title
    if any(tag in solar_tags and (word in solar_words) for word, tag in pos_tags):
        return True
    elif any(word in solar_words for word in keywords):
        return True
    return False

# Apply the 'is_solar' function to each value in the 'title' column and save the results to a new column called 'is_solar'
circles['is_solar'] = circles['title'].apply(is_solar)


In [ ]:
circles[circles['is_solar']].head()

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar
48390,48390,2c9a6af6-3a90-44bb-bd63-ee1c45a528ae,John Burko Sun power,114.0,False,[],False,"[Sun, power]",True
50612,50612,89054214-2e71-4a86-b375-c3ab8a5a5f1b,1801-sun 3rd,39.0,False,[],False,[sun],True
63489,63489,00005504-dde1-3901-684c-63961c710002,"March 14 - ""sun"" Day",31.0,False,[],False,[sun],True
68037,68037,000058d8-6003-d551-c07b-7c42f10a0002,#La solar - Vicente Garcia,14.0,False,[],False,[solar],True
70460,70460,0000592c-c453-d551-c01e-66791bfa0002,Lucas.com/power,8.0,False,[],False,[power],True


In [ ]:
circles['solar_keywords'].apply(tuple).unique()

array([(), ('Power',), ('Sun',), ('Panel',), ('System',), ('SOLAR',),
       ('Energy',), ('POWER',), ('SUN',), ('GRID',), ('Solar',),
       ('Sun', 'power'), ('sun',), ('Grid',), ('solar',), ('power',),
       ('Electricity',), ('SYSTEM',), ('PANEL',), ('Solar', 'System'),
       ('energy',), ('Sun', 'Power'), ('Sun', 'Energy'), ('panel',),
       ('POWER', 'PANEL'), ('Renewable', 'Energy'), ('Solar', 'Panel'),
       ('Sun', 'Sun'), ('GRID', 'PHOTOVOLTAIC'), ('system',), ('grid',),
       ('eNeRgY',), ('ENERGY',), ('SUn',), ('Solar', 'Power'),
       ('EnerGy',), ('Solar', 'Sun'), ('Renewable',), ('Solar', 'panel'),
       ('solar', 'system'), ('Power', 'System')], dtype=object)

In [ ]:
len(circles[circles['is_solar']])

114

In [ ]:
round(len(circles[circles['is_solar']])/len(circles)*100,2)

0.01

In [ ]:
circles[circles['is_solar']]['media_count'].sum()

9462.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_solar']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.0

#**Add Event Tags**

In [ ]:
# Define a list of part-of-speech tags for event-related words
event_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Define a list of event-related words
event_words = ['event', 'concert', 'festival', 'performance', 'show', 'party', 'celebration', 'ceremony', 'gala', 'conference', 'holiday']

# Define the regular expression pattern to extract event-related keywords
pattern = r'\b(?:%s)\b' % '|'.join(event_words)

# Extract event-related keywords from the 'title' column and save to a new column called 'event_keywords'
circles['event_keywords'] = circles['title'].str.findall(pattern, flags=re.IGNORECASE)

# Define a function to check if a string contains event-related tags or keywords
def is_event(title):
    # Extract event-related keywords from the title
    keywords = re.findall(pattern, title, flags=re.IGNORECASE)
    # Tokenize the title and tag the parts of speech
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    # Check if any event-related tags or keywords are present in the title
    if any(tag in event_tags and (word in event_words) for word, tag in pos_tags):
        return True
    elif any(word in event_words for word in keywords):
        return True
    return False

# Apply the 'is_event' function to each value in the 'title' column and save the results to a new column called 'is_event'
circles['is_event'] = circles['title'].apply(is_event)

In [ ]:
len(circles[circles['is_event']])

3887

In [ ]:
round(len(circles[circles['is_event']])/len(circles)*100,2)

0.21

In [ ]:
circles[circles['is_event']]['media_count'].sum()

462208.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_event']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.14

In [ ]:
circles['event_keywords'].apply(tuple).unique()

array([(), ('Party',), ('celebration',), ('party',), ('Festival',),
       ('Holiday',), ('GALA',), ('Celebration',), ('Ceremony',),
       ('show',), ('Show',), ('Concert',), ('Party', 'Celebration'),
       ('PARTY',), ('Conference',), ('Event',), ('Gala',),
       ('Holiday', 'Party'), ('festival',), ('holiday',), ('concert',),
       ('Holiday', 'Celebration'), ('event',), ('SHOW',),
       ('Performance',), ('Holiday', 'Gala'), ('CONFERENCE',),
       ('party', 'party'), ('conference',), ('HOLIDAY', 'PARTY'),
       ('Party', 'Party'), ('FESTIVAL',), ('ceremony',),
       ('Holiday', 'Show'), ('Holiday', 'party'), ('CELEBRATION',),
       ('Show', 'Show'), ('Holiday', 'Concert'), ('performance',),
       ('HOLIDAY',), ('Party', 'Show'), ('CONCERT',), ('Gala', 'Concert'),
       ('Event', 'Event'), ('EVENT',), ('Celebration', 'Party'),
       ('party', 'party', 'party'), ('Holiday', 'Event'),
       ('holiday', 'Holiday'), ('gala',), ('holiday', 'show'),
       ('celebration', 'Cer

In [ ]:
circles[circles['is_event']].head(3)

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event
93,93,00005c22-b096-d551-c026-3d4be4720002,Christmas Day 2018 - Enrile celebration,35.0,False,[],False,[],False,[celebration],True
199,199,32d3371c-bed8-413b-b1a7-510d7c370a51,Birthday party!!!,124.0,False,[],False,[],False,[party],True
582,582,00005deb-360b-f8d1-c166-866926b40002,2019 Christmas party slideshow,197.0,False,[],False,[],False,[party],True


# **Add Professional Career Tags**

In [ ]:
# Define a list of part-of-speech tags for occupation-related words
occupation_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Define a list of occupation-related words
occupation_words = ['job', 'work', 'career', 'profession', 'occupation', 'employment', 'position', 'vocation', 'trade', 'field']

# Define the regular expression pattern to extract occupation-related keywords
pattern = r'\b(?:%s)\b' % '|'.join(occupation_words)

# Extract occupation-related keywords from the 'title' column and save to a new column called 'occupation_keywords'
circles['occupation_keywords'] = circles['title'].str.findall(pattern, flags=re.IGNORECASE)

# Define a function to check if a string contains occupation-related tags or keywords
def is_occupation(title):
    # Extract occupation-related keywords from the title
    keywords = re.findall(pattern, title, flags=re.IGNORECASE)
    # Tokenize the title and tag the parts of speech
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    # Check if any occupation-related tags or keywords are present in the title
    if any(tag in occupation_tags and (word in occupation_words) for word, tag in pos_tags):
        return True
    elif any(word in occupation_words for word in keywords):
        return True
    return False

# Apply the 'is_occupation' function to each value in the 'title' column and save the results to a new column called 'is_occupation'
circles['is_occupation'] = circles['title'].apply(is_occupation)


In [ ]:
circles[circles['is_occupation']].head()

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation
14812,14812,0bb0e7aa-5e1a-4c40-8ba9-ebc43319bf70,Sunflower field 2021 0728,60.0,False,[],False,[],False,[],False,[field],True
22045,22045,00005ac7-e242-d551-c01e-36e3cb720002,Jillianne’s half day spent at work...,3.0,False,[],False,[],False,[],False,[work],True
34443,34443,00005b1e-0625-d551-c01d-9f1fc65d0002,mcdonalds summer job,29.0,False,[],False,[],False,[],False,[job],True
34925,34925,dd6c1b0d-193e-4e37-a19c-d937368f1b7f,kappa work week,1141.0,True,[],False,[],False,[],False,[work],True
40095,40095,9eac4fbe-11c5-4100-ad38-60d3ca68da7a,William Kramer job-99430 SR-837707,33.0,False,[],False,[],False,[],False,[job],True


In [ ]:
len(circles[circles['is_occupation']])

406

In [ ]:
round(len(circles[circles['is_occupation']])/len(circles)*100,2)

0.02

In [ ]:
circles[circles['is_occupation']]['media_count'].sum()

55678.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_occupation']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.02

In [ ]:
circles['occupation_keywords'].apply(tuple).unique()

array([(), ('Field',), ('Job',), ('Work',), ('JOB',), ('Trade',),
       ('field',), ('Career',), ('work',), ('Position',), ('WORK',),
       ('job',), ('Employment',), ('Work', 'Work'),
       ('Work', 'Work', 'Work', 'Work', 'Work'), ('CAREER',), ('FIELD',),
       ('Vocation',), ('work', 'work', 'work'), ('trade',),
       ('Work', 'Work', 'Work'), ('Profession',), ('career',), ('JoB',),
       ('Job', 'Job'), ('work', 'field'), ('Career', 'Field'),
       ('Work', 'Job'), ('Field', 'Career'), ('Field', 'Work'),
       ('Work', 'work'), ('work', 'Work', 'Work', 'Work', 'Work', 'Work'),
       ('Work', 'Field'), ('Field', 'Field'), ('WORK', 'WORK'),
       ('Job', 'JOB'), ('FIeld',), ('position',), ('JOB', 'JOB'),
       ('Job', 'Field'), ('Occupation',), ('TRADE',)], dtype=object)

#**Add Animals Tags**

In [ ]:
# Define a list of part-of-speech tags for pet or animal-related words
animal_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ']

# Define a list of pet or animal-related words
animal_words = ['pet', 'animal', 'dog', 'cat', 'fish', 'bird', 'hamster', 'turtle', 'snake', 'rabbit',
                'mouse', 'guinea pig', 'ferret', 'chinchilla', 'hedgehog', 'parrot', 'canary', 'cockatiel', 'iguana',
                'gecko', 'tarantula', 'scorpion', 'bee', 'butterfly', 'spider', 'ladybug', 'ant', 'grasshopper',
                'cricket', 'dragonfly', 'snail', 'slug', 'worm', 'leech', 'octopus', 'squid', 'jellyfish', 'seahorse',
                'shark', 'whale', 'dolphin', 'sea turtle', 'penguin', 'polar bear', 'elephant', 'giraffe', 'lion',
                'tiger', 'bear', 'gorilla', 'monkey', 'zebra', 'horse', 'cow', 'pig', 'sheep', 'goat', 'chicken',
                'duck', 'turkey', 'bee', 'insect', 'arachnid', 'crustacean', 'mollusk']

# Define the regular expression pattern to extract pet or animal-related keywords
pattern = r'\b(?:%s)\b' % '|'.join(animal_words)

# Extract pet or animal-related keywords from the 'title' column and save to a new column called 'animal_keywords'
circles['animal_keywords'] = circles['title'].str.findall(pattern, flags=re.IGNORECASE)

# Define a function to check if a string contains pet or animal-related tags or keywords
def is_animal(title):
    # Extract pet or animal-related keywords from the title
    keywords = re.findall(pattern, title, flags=re.IGNORECASE)
    # Tokenize the title and tag the parts of speech
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    # Check if any pet or animal-related tags or keywords are present in the title
    if any(tag in animal_tags and (word in animal_words) for word, tag in pos_tags):
        return True
    elif any(word in animal_words for word in keywords):
        return True
    return False

# Apply the 'is_animal' function to each value in the 'title' column and save the results to a new column called 'is_animal'
circles['is_animal'] = circles['title'].apply(is_animal)


In [ ]:
circles[circles['is_animal']]

,circle_uuid,title,media_count,has_greek_words,destination_names,people_names,places,sports,nature,dates,...,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation,animal_keywords,is_animal
1336,da0add50-3f74-4952-80c3-fd8e387d9ed7,alpha chi chicken thighs,7635,True,alpha,[],[],[],[],[],...,[],False,[],False,[],False,[],False,[chicken],True
9227,7233c9d1-069a-4a1d-99dd-5e9bdd2b4eaa,fuck go fish,2592,False,fuck,[],[],[],[],[],...,[],False,[],False,[],False,[],False,[fish],True
17836,109a270d-431c-405e-9619-93e17bb5b540,Roadtrip and New york chicken strips,1746,False,Roadtrip,[],['New york'],[],[],[],...,[],False,[],False,[],False,[],False,[chicken],True
22670,00005711-32b8-d551-c04c-b2b73c190002,lee animal farm,1514,False,lee,['lee animal farm'],[],[],[],[],...,[],False,[],False,[],False,[],False,[animal],True
23130,cb4d2730-e110-49d9-a151-7df7f383c253,penguin pals,1495,False,penguin,[],[],[],[],[],...,[],False,[],False,[],False,[],False,[penguin],True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204496,000055f7-212a-d170-be37-97f114870002,dog,42,False,dog,[],[],[],[],[],...,[],False,[],False,[],False,[],False,[dog],True
1204735,58e9d4a5-3fd5-438e-bed0-75f983635aa9,Dawson's 11th bday & goat visit 2022,42,False,Dawson,[],['Dawson'],[],[],['2022'],...,[],False,[],False,[],False,[],False,[goat],True
1213851,00005dde-edd9-f8d1-c10d-108a5cd60002,turkey day,41,False,turkey,[],[],[],[],[],...,[],False,[],False,[],False,[],False,[turkey],True
1216874,db568840-9dad-4176-be8a-c9a1b7eaee20,12b zebra celebration,41,False,zebra,[],[],[],[],[],...,[],False,[],False,[celebration],True,[],False,[zebra],True


In [ ]:
len(circles[circles['is_animal']])

570

In [ ]:
round(len(circles[circles['is_animal']])/len(circles)*100,2)

0.03

In [ ]:
circles[circles['is_animal']]['media_count'].sum()

84834.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_animal']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.02

In [ ]:
circles['animal_keywords'].apply(tuple).unique()

array([(), ('Shark',), ('Fish',), ('CAT',), ('Animal',), ('Horse',),
       ('Bear',), ('Bird',), ('Pig',), ('Ferret',), ('Turkey',),
       ('bird',), ('Pet',), ('Goat',), ('Tiger',), ('Mouse',),
       ('animal',), ('GOAT',), ('Ant',), ('Chicken',), ('Cat',), ('Dog',),
       ('Jellyfish',), ('Polar Bear',), ('Monkey',), ('DOG',),
       ('TURKEY',), ('Lion',), ('Hedgehog',), ('Duck', 'Duck'),
       ('Snake',), ('Sheep',), ('Whale',), ('BUTTERFLY',), ('Bee',),
       ('bear',), ('Duck',), ('PET',), ('CoW',), ('tiger',),
       ('Elephant',), ('Rabbit',), ('SPIDER',), ('cow',),
       ('Bear', 'Bear'), ('Bear', 'Mouse'), ('Cow',), ('duck',), ('dog',),
       ('horse',), ('BEAR',), ('BEE',), ('chicken',), ('Penguin',),
       ('Butterfly',), ('cat',), ('Worm',), ('Turtle',), ('lion',),
       ('Octopus',), ('SNAKE',), ('Cricket',), ('Giraffe',), ('Dolphin',),
       ('fish',), ('goat',), ('DOLPHIN',), ('snake',), ('Zebra',),
       ('ELEPHANT',), ('pig',), ('Spider',), ('SHARK',), ('L

# **Add Travel Tags**

In [ ]:
# Define a list of part-of-speech tags for travel-related words
travel_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ']

# Define a list of travel-related words
travel_words = [
    'travel', 'trip', 'tour', 'vacation', 'adventure',
    'journey', 'expedition', 'cruise', 'getaway', 'holiday',
    'exploration', 'pilgrimage', 'excursion', 'voyage', 'itinerary',
    'destination', 'wanderlust', 'safari', 'trek', 'backpacking',
    'camping', 'road trip', 'beach', 'mountains', 'island',
    'city break', 'culture', 'history', 'architecture', 'food',
    'wine', 'beer', 'festivals', 'music', 'art', 'nature', 'wildlife'
]

# Define the regular expression pattern to extract destination names
pattern = r'(\b[A-Z][a-z]+(?:[ -][A-Z][a-z]+)*\b|\b[a-z]+(?:[ -][A-Z][a-z]+)*\b)'

# Extract destination names from the 'title' column and save to a new column called 'destination_names'
circles['destination_names'] = circles['title'].str.extract(pattern, expand=False)

# Define a function to check if a string contains travel-related tags or words
def is_travel(title):
    destination_names = re.findall(pattern, title)
    for name in destination_names:
        # Tokenize the destination name and tag the parts of speech
        tokens = nltk.word_tokenize(name)
        pos_tags = nltk.pos_tag(tokens)
        # Check if any travel-related tags or words are present in the destination name
        if any(tag in travel_tags and (word in travel_words or 'travel' in word) for word, tag in pos_tags):
            return True
    return False

# Apply the 'is_travel' function to each value in the 'title' column and save the results to a new column called 'is_travel'
circles['is_travel'] = circles['title'].apply(is_travel)

In [ ]:
circles[circles['is_travel']].head()

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation,animal_keywords,is_animal,destination_names,is_travel
151,151,0000538f-72b6-3dab-0506-e16318650002,Kauai trip 2014,45.0,False,[],False,[],False,[],False,[],False,[],False,Kauai,True
560,560,00005bc1-4d97-d551-c03b-946e02410002,Smoky mountains,186.0,False,[],False,[],False,[],False,[],False,[],False,Smoky,True
577,577,00005e3c-dcfa-78bb-8958-02f5a82c0002,Gatlinburg trip 2020,35.0,False,[],False,[],False,[],False,[],False,[],False,Gatlinburg,True
604,604,87b088d5-ce76-4495-b727-a6a8f52e5e5c,Kenzie's 21st birthday cruise,276.0,False,[],False,[],False,[],False,[],False,[],False,Kenzie,True
844,844,4b206189-ff4f-4044-bf26-b99cea87414f,Napa girls trip 2022,726.0,False,[],False,[],False,[],False,[],False,[],False,Napa,True


In [ ]:
len(circles[circles['is_travel']])

8195

In [ ]:
round(len(circles[circles['is_travel']])/len(circles)*100,2)

0.44

In [ ]:
circles[circles['is_travel']]['media_count'].sum()

1909054.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_travel']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.56

In [ ]:
circles_destination_names = list(circles['destination_names'].dropna().unique())
circles_destination_names = pd.DataFrame(circles_destination_names, columns=['destination_names'])

In [ ]:
circles_destination_names.head()

,destination_names
0,State Fair
1,New
2,Hemi
3,Archer
4,Hannah


In [ ]:
#circles_destination_names.to_csv('circles_destination_names.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#circles_destination_names.to_csv('/content/drive/MyDrive/PhotoCircle/circles_destination_names.csv', index=False)

# **Add Locations Tags**

In [ ]:
# Define a list of part-of-speech tags for location-related words
location_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ']

# Define a list of location-related words
location_words = ['city', 'county', 'state', 'country', 'province', 'territory', 'region', 
                  'town', 'village', 'hamlet', 'metropolis', 'municipality', 'district', 'zone', 
                  'neighborhood', 'suburb', 'countryside', 'coast', 'mountains', 'desert', 'island', 
                  'peninsula', 'valley', 'canyon', 'plateau', 'river', 'lake', 'ocean', 'sea', 'bay', 
                  'harbor', 'port', 'airport', 'station']

# Define the regular expression pattern to extract location names
pattern = r'(\b[A-Z][a-z]+(?:[ -][A-Z][a-z]+)*\b|\b[a-z]+(?:[ -][A-Z][a-z]+)*\b)'

# Extract location names from the 'title' column and save to a new column called 'location_names'
circles['location_names'] = circles['title'].str.extract(pattern, expand=False)

# Define a function to check if a string contains location-related tags or words
def is_location(title):
    location_names = re.findall(pattern, title)
    for name in location_names:
        # Tokenize the location name and tag the parts of speech
        tokens = nltk.word_tokenize(name)
        pos_tags = nltk.pos_tag(tokens)
        # Check if any location-related tags or words are present in the location name
        if any(tag in location_tags and (word in location_words or 'location' in word or 'place' in word) for word, tag in pos_tags):
            return True
    return False

# Apply the 'is_location' function to each value in the 'title' column and save the results to a new column called 'is_location'
circles['is_location'] = circles['title'].apply(is_location)

In [ ]:
circles[circles['is_location']].head()

,Unnamed: 0,circle_uuid,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation,animal_keywords,is_animal,destination_names,is_travel,location_names,is_location
58,58,00005e35-9d7a-78bb-8958-02dbdad30002,Lost valley Tree Project,134.0,False,[],False,[],False,[],False,[],False,[],False,Lost,False,Lost,True
138,138,1d6366ad-ec9f-4269-9ac6-8a62efcdefd2,Aj And The Kids Visit To carson city 7/2021,40.0,False,[],False,[],False,[],False,[],False,[],False,Aj And The Kids Visit To,False,Aj And The Kids Visit To,True
255,255,5ee5f9f3-2483-4ab8-b9b0-b835f2195b2c,Bills lake Cottage Weekend,141.0,False,[],False,[],False,[],False,[],False,[],False,Bills,False,Bills,True
332,332,6878fcb6-d15c-4adc-8675-c39cf66bbdee,Tennis state,24.0,False,[],False,[],False,[],False,[],False,[],False,Tennis,False,Tennis,True
444,444,2a1c3668-0e4a-43e6-824d-16ff92c8c68a,Christmas village 2022,387.0,False,[],False,[],False,[],False,[],False,[],False,Christmas,False,Christmas,True


In [ ]:
len(circles[circles['is_location']])

3200

In [ ]:
round(len(circles[circles['is_location']])/len(circles)*100,2)

0.17

In [ ]:
circles[circles['is_location']]['media_count'].sum()

479786.0

In [ ]:
circles['media_count'].sum()

341813160.0

In [ ]:
round(circles[circles['is_location']]['media_count'].sum()/circles['media_count'].sum()*100,2)

0.14

In [ ]:
circles_location_names = list(circles['location_names'].dropna().unique())
circles_location_names = pd.DataFrame(circles_location_names, columns=['location_names'])

In [ ]:
#circles_location_names.to_csv('/content/drive/MyDrive/PhotoCircle/circles_location_names.csv', index=False)

# **Save Tags to csv**

In [ ]:
circles.to_csv('circles_tags.csv')

In [ ]:
#circles.to_csv('/content/drive/MyDrive/PhotoCircle/circles_with_tags.csv', index=False)

#**Merge users and circles with circle_membership**

In [ ]:
circle_membership = pd.read_csv('circle_membership.csv')

In [ ]:
circle_membership.shape

(10578744, 4)

In [ ]:
# Left join circle_membership with circles on circle_uuid
merged_circles = pd.merge(circle_membership, circles, on='circle_uuid', how='left')

In [ ]:
merged_circles.shape

(10578744, 22)

In [ ]:
# Left join merged_circles with users on user_uuid
merged_data = pd.merge(merged_circles, users_gender, on='user_uuid', how='left')

In [ ]:
merged_data.shape

(10578744, 32)

In [ ]:
merged_data.head(3)

,Unnamed: 0_x,circle_uuid,user_uuid,media_added_count_x,Unnamed: 0_y,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation,animal_keywords,is_animal,destination_names,is_travel,location_names,is_location,Unnamed: 0,display_name,email_domain,active_count_in_last_30_days,media_added_count_y,first_name,num_female,num_male,female_pct,is_student
0,0,00005962-836d-d551-c07b-5c2040500002,00000000-575c-c42a-d551-c018b1942652,96,213173,Georgia Jean Williams,160.0,False,[],False,[],False,[],False,[],False,[],False,Georgia Jean Williams,False,Georgia Jean Williams,False,3192531.0,Darwin Williams,stthomas.edu,6,15.0,Darwin,83.0,24680.0,0.34,True
1,1,00005bfc-e475-d551-c026-3d658eab0002,4d4f4f68-feff-4490-bd45-ae90ba4acdbf,6,1270287,Tri Delta,12235.0,True,[],False,[],False,[],False,[],False,[],False,Tri Delta,False,Tri Delta,False,1910893.0,tara ghimenti,gmail.com,0,0.0,tara,0.0,0.0,50.00,False
2,2,55a82813-15c0-4faf-897d-d04803036af0,c31e9b29-bf13-4493-a017-54d531d51d7b,31,390753,Vegas 2022,31.0,False,[],False,[],False,[],False,[],False,[],False,Vegas,False,Vegas,False,2607998.0,Brenda Yorks,privaterelay.appleid.com,0,0.0,Brenda,607096.0,2121.0,99.65,False


In [ ]:
merged_data = merged_data.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0'], axis=1)

In [ ]:
merged_data.head(3)

,circle_uuid,user_uuid,media_added_count_x,title,media_count,has_greek_words,real_estate_keywords,is_real_estate,solar_keywords,is_solar,event_keywords,is_event,occupation_keywords,is_occupation,animal_keywords,is_animal,destination_names,is_travel,location_names,is_location,display_name,email_domain,active_count_in_last_30_days,media_added_count_y,first_name,num_female,num_male,female_pct,is_student
0,00005962-836d-d551-c07b-5c2040500002,00000000-575c-c42a-d551-c018b1942652,96,Georgia Jean Williams,160.0,False,[],False,[],False,[],False,[],False,[],False,Georgia Jean Williams,False,Georgia Jean Williams,False,Darwin Williams,stthomas.edu,6,15.0,Darwin,83.0,24680.0,0.34,True
1,00005bfc-e475-d551-c026-3d658eab0002,4d4f4f68-feff-4490-bd45-ae90ba4acdbf,6,Tri Delta,12235.0,True,[],False,[],False,[],False,[],False,[],False,Tri Delta,False,Tri Delta,False,tara ghimenti,gmail.com,0,0.0,tara,0.0,0.0,50.00,False
2,55a82813-15c0-4faf-897d-d04803036af0,c31e9b29-bf13-4493-a017-54d531d51d7b,31,Vegas 2022,31.0,False,[],False,[],False,[],False,[],False,[],False,Vegas,False,Vegas,False,Brenda Yorks,privaterelay.appleid.com,0,0.0,Brenda,607096.0,2121.0,99.65,False


In [ ]:
merged_data.to_csv('/content/drive/MyDrive/PhotoCircle/photocircle.csv', index=False)

In [ ]:
len(circle_membership['circle_uuid'].unique())

2042587

In [ ]:
len(circles['circle_uuid'].unique())

1861528

In [ ]:
# Get the circle_uuid and user_uuid from circles and users that are not in circle_membership
circles_not_in_membership = circles[~circles['circle_uuid'].isin(merged_data['circle_uuid'])]['circle_uuid']
users_not_in_membership = users[~users['user_uuid'].isin(merged_data['user_uuid'])]['user_uuid']

In [ ]:
circles_not_in_membership

237973     21
458642      8
1724576    91
Name: circle_uuid, dtype: object

In [ ]:
users_not_in_membership.shape

(112705,)